In [1]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 11.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 17.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 9.8 MB/s eta 0:00:00:00:0100:01

In [2]:
import os

# Force Python to only see the first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
from datasets import load_dataset, ClassLabel, Value
import yaml


def load_prompts(yaml_path="prompts.yaml"):
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

def transform_dataset_task(dataset, task, technique, tokenizer, yaml_path="/configs/prompts.yaml"):
    """
    Transforms the dataset by applying the specific prompt technique from the YAML.
    
    Args:
        dataset (DatasetDict or Dataset): The Hugging Face dataset.
        task (str): 'task_1_clarity' or 'task_2_evasion'.
        technique (str): 'default', 'persona', 'chain_of_thought', 'definition_aware'.
        yaml_path (str): Path to prompts.yaml.
        
    Returns:
        Dataset: The dataset with a new 'text' column containing the formatted prompt.
    """
    prompts = load_prompts(yaml_path)

    # Validation
    if task not in prompts:
        raise ValueError(f"Task '{task}' not found in yaml. Available: {list(prompts.keys())}")
    if technique not in prompts[task]:
        raise ValueError(f"Technique '{technique}' not found for {task}. Available: {list(prompts[task].keys())}")
        
    config = prompts[task][technique]
    system_prompt = config['system']
    
    if task == 'task_1_clarity':
        label_column = "clarity_label"
        unique_labels = sorted(set(dataset["train"][label_column]))
    elif task == 'task_2_evasion':
        label_column = "evasion_label"
        unique_labels = sorted(set(dataset["train"][label_column]))
    # Step 1: Temporarily cast to ClassLabel for stratification
    
    dataset = dataset['train'].cast_column(
        label_column,
        ClassLabel(names=unique_labels)
    )
    # Step 2: Stratified split
    dataset = dataset.train_test_split(
        test_size=0.1,
        stratify_by_column=label_column,
        seed=42,
    )
    dataset = dataset['test']

    # Step 3: Convert labels BACK to strings
    def convert_label_back(example):
        idx = int(example[label_column])   # convert "4" → 4
        example[label_column] = unique_labels[idx]
        return example

    dataset = dataset.cast_column(label_column, Value("string"))
    dataset = dataset.map(convert_label_back)
        
    def format_row(row):
        # 1. Extract inputs
        q = row['interview_question']
        a = row['interview_answer']
        p = row['president']
        d = row['date']
        raw_label = row[label_column]
        
    
        # 2. Format the full string
        # Note: We strip whitespace to prevent tokenization issues at boundaries
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"President: {p}\nDate: {d}\n{q}\n{a}"}, #Q, A are already in the dataset before each Question and Answer respectively
            {"role": "assistant", "content": str(raw_label)}
            ]
        
        # 3. Apply Template
        # tokenize=False gives you the formatted string back so you can debug it
        # add_generation_prompt=False because we include the assistant response (training)
        formatted_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        
        return {"text": formatted_text}

    # Apply transformation
    # remove_columns is optional but recommended if you only need the 'text' for training
    return dataset.map(format_row, remove_columns=dataset.column_names)

In [4]:
from unsloth import FastLanguageModel
import torch

def get_model_and_tokenizer(model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit", max_seq_length=2048,
                            dtype=None, load_in_4bit = True, output_dir='.'):
    """
    Loads the 4-bit Quantized Mistral model with Unsloth optimizations.
    Returns: (model, tokenizer) tuple
    """
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,    # Force 4-bit loading
        dtype=dtype # Explicitly pass torch_dtype to address KeyError
    )

    # Attach LoRA Adapters (Configuration)
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha = 16,
        lora_dropout = 0,       # Optimized for Unsloth
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 3407,
    )
    model.save_pretrained(output_dir)
    return model, tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-02-15 21:18:02.443274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771190282.835007      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771190282.957932      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771190283.905004      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771190283.905038      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771190283.905041      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
from unsloth.chat_templates import get_chat_template

model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # Forces Mistral v0.3 format
    )

In [28]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_2_evasion', 'chain_of_thought', tokenizer, "/kaggle/input/datasets/ostriftis/prompts1/prompts.yaml")

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [29]:
print(subset[0])

{'text': "<s>[INST] First, explain whether the politician answers directly or avoids the question. Think step by step, but keep your reasoning separate from the final answer.\nThen, classify the evasion level of the answer as 'Explicit', 'Implicit', 'Dodging', 'General', 'Deflection', 'Partial/half-answer', 'Declining to answer', 'Claims ignorance', and 'Clarification'.\n President: Barack Obama\nDate: August 01, 2014\nQ. Mr. President, like that cease-fire, you've called for diplomatic solutions not only in Israel and Gaza, but also in Ukraine, in Iraq, to very little effect so far. Has the United States of America lost its influence in the world? Have you lost yours?\nYes. Look, this is a common theme that folks bring up. Apparently, people have forgotten that America, as the most powerful country on Earth, still does not control everything around the world. And so our diplomatic efforts often take time. They often will see progress and then a step backwards. That's been true in the 

In [8]:
del dataset

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


def train_model(model, tokenizer, dataset, output_dir='lora.pth', 
                training_args = TrainingArguments(
                    per_device_train_batch_size = 1,
                    gradient_accumulation_steps = 8,
                    warmup_steps = 5,
                    num_train_epochs = 1, # Set this for 1 full training run.
                    learning_rate = 2e-4,
                    fp16 = not is_bfloat16_supported(),
                    bf16 = is_bfloat16_supported(),
                    logging_steps = 1,
                    optim = "adamw_8bit",
                    weight_decay = 0.01,
                    lr_scheduler_type = "linear",
                    seed = 21,
                    output_dir = "outputs",
                    report_to = "none", # Use this for WandB etc
                    )
                ):
    """
    Executes the training loop.
    Args:
        model: Pre-loaded Unsloth model object
        tokenizer: Pre-loaded Unsloth tokenizer
        output_dir: Where to save the adapter
        subset_size: Limit for pilot testing
    """
        
    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset,
        dataset_text_field = "text",
        max_seq_length = 2048,
        dataset_num_proc = 2,
        packing = False,
        args = training_args,
    )
    torch.cuda.empty_cache()

    trainer.train()
    print(f"Saving adapter to {output_dir}")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [31]:
train_model(model, tokenizer, dataset=subset)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/345 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.525900
2,2.617900
3,2.459800
4,2.321800
5,2.058100
6,2.091600
7,1.911400
8,1.785600
9,1.714600
10,1.715900


Saving adapter to lora.pth


In [32]:
import shutil
from IPython.display import FileLink

# 1. Compress the lora.pth folder into a single zip file
shutil.make_archive('lora_weights', 'zip', '/kaggle/working/lora.pth')

# 2. Generate a clickable download link for the zipped file
FileLink(r'lora_weights.zip')

/kaggle/working/lora_weights.zip

In [48]:
del model, tokenizer

In [49]:
torch.cuda.empty_cache()

In [50]:
model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [51]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_2_evasion', 'default', tokenizer, "/kaggle/input/datasets/ostriftis/prompts1/prompts.yaml")

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [52]:
train_model(model, tokenizer, dataset=subset, output_dir='task2_default')

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/345 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.430000
2,2.520200
3,2.394700
4,2.381000
5,2.160700
6,2.257400
7,2.177100
8,2.089500
9,2.070100
10,2.004500


Saving adapter to task2_default


In [54]:
# 1. Compress the lora.pth folder into a single zip file
shutil.make_archive('lora_weights', 'zip', '/kaggle/working/task2_default')

# 2. Generate a clickable download link for the zipped file
FileLink(r'lora_weights.zip')

/kaggle/working/lora_weights.zip

In [64]:
del model, tokenizer
torch.cuda.empty_cache()

In [65]:
model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [66]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_2_evasion', 'persona', tokenizer, "/kaggle/input/datasets/ostriftis/prompts1/prompts.yaml")

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [68]:
train_model(model, tokenizer, dataset=subset, output_dir='task2_persona')

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/345 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.430000
2,2.520200
3,2.394800
4,2.381000
5,2.160700
6,2.257500
7,2.177100
8,2.089700
9,2.070100
10,2.004400


Saving adapter to task2_persona


In [69]:
shutil.make_archive('task2_persona', 'zip', '/kaggle/working/task2_persona')

# 2. Generate a clickable download link for the zipped file
FileLink(r'task2_persona.zip')

/kaggle/working/task2_persona.zip

In [24]:
del model, tokenizer
torch.cuda.empty_cache()

In [ ]:
model, tokenizer = get_model_and_tokenizer()

In [11]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_2_evasion', 
                                'definition_aware', tokenizer, "/kaggle/input/datasets/ostriftis/prompts1/prompts.yaml")

Casting the dataset:   0%|          | 0/345 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [12]:
train_model(model, tokenizer, dataset=subset, output_dir= "task2_definition")

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/345 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.430500
2,2.510900
3,2.298200
4,2.117200
5,1.859800
6,1.820500
7,1.591200
8,1.463000
9,1.363200
10,1.413800


Saving adapter to task2_definition


In [15]:
import shutil
from IPython.display import FileLink

shutil.make_archive('task2_definition', 'zip', '/kaggle/working/task2_definition')

# 2. Generate a clickable download link for the zipped file
FileLink(r'task2_definition.zip')

/kaggle/working/task2_definition.zip

In [12]:
from datasets import load_dataset
dataset = load_dataset("ailsntua/QEvasion")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

In [12]:
train_dataset = dataset['train']

In [16]:
from tqdm import tqdm
from collections import Counter

def predict_label(question, answer, president, date, task_type, technique, model, tokenizer):
    """Generates a classification label for a given question/answer pair."""
    
    # Set the correct system prompt based on the task
    prompts = load_prompts("/kaggle/input/datasets/ostriftis/prompts1/prompts.yaml")
    
    sys_prompt = prompts[task_type][technique]
    messages = [
        {"role": "system", "content": sys_prompt['system']},
        {"role": "user", "content": f"President: {president}\nDate: {date}\n{question}\n{answer}"},
    ]

    # Tokenize and format with the chat template
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048,
    ).to("cuda")
    # Generate the classification
    outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],  # CRITICAL: Pass the attention mask
            max_new_tokens=50,  # Reduced since we expect short labels
            do_sample=False,
            temperature=1.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    
    # Slice the output tensor to isolate only the newly generated tokens
    input_length = inputs["input_ids"].shape[1]
    new_tokens = outputs[0][input_length:]


    # Slice the output tensor to isolate only the newly generated tokens
    # This is much cleaner and less error-prone than string splitting!
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()


def evaluate_model_on_dataset(test_dataset, task_type, technique, model, tokenizer):
    """Iterates through the test dataset and collects predictions."""
    results = []
    task_mapping = {'Clear Reply': ['Explicit'],
                  'Ambivalent': ['Implicit', 'Dodging', 'General', 'Deflection', 'Partial/half-answer'],
                  'Clear Non-Reply': ['Declining to answer', 'Claims ignorance', 'Clarification']}
    
    for row in tqdm(test_dataset):
        question = row['interview_question']
        answer = row['interview_answer']
        clarity = row['clarity_label']
        president = row['president']
        date = row['date']
        ground_truth = None
        if task_type == 'task_1_clarity':
            ground_truth = clarity
        else:
            valid_evasions = task_mapping.get(clarity, [])
                
            # Gather the 3 annotations
            raw_votes = [row['annotator1'], row['annotator2'], row['annotator3']]
            
            # Filter step: only keep votes that exist in the valid mapping
            valid_votes = [vote for vote in raw_votes if vote in valid_evasions]
            
            if valid_votes == []:
                # Fallback for Zero Valid Votes: Pick the most common evasion tactic for that clarity class
                fallbacks = {   
                    'Clear Reply': 'Explicit',
                    'Ambivalent': 'Dodging', 
                    'Clear Non-Reply': 'Declining to answer'
                }
                evasion = fallbacks.get(clarity, None)
            else: 
                vote_counts = Counter(valid_votes)
                most_common = vote_counts.most_common()
                
                evasion = most_common[0][0]
                
            ground_truth = evasion

        prediction = predict_label(question, answer, president, date, task_type, technique, model, tokenizer)
        
        results.append({
            "question": question,
            "answer": answer,
            "ground_truth": ground_truth,
            "prediction": prediction
        })
        
    return results


In [17]:
del model

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# 2. Load your different trained adapters
# model.load_adapter("/kaggle/working/lora.pth", adapter_name="v2_cot")
# 3. Swap instantly to test


==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_lay

In [57]:
model.load_adapter("/kaggle/working/task2_default", adapter_name="v2_default")

In [62]:
model.set_adapter("v2_default")
res['v2_default'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_2_evasion',
                                              technique='default', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:36<00:00,  1.09s/it]


In [22]:
test_dataset = dataset['test']

In [46]:
res = {}
model.set_adapter("v2_cot")
res['v2_cot'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_2_evasion', 
                                          technique = 'chain_of_thought',model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:43<00:00,  1.12s/it]


In [72]:
model.load_adapter("/kaggle/working/task2_persona", adapter_name="v2_persona")

In [73]:
model.set_adapter("v2_persona")
res['v2_persona'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_2_evasion',
                                              technique='persona', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:45<00:00,  1.12s/it]


In [19]:
model.load_adapter("/kaggle/working/task2_definition", adapter_name="v2_definition")

In [20]:
res = {}

In [23]:
model.set_adapter("v2_definition")
res['v2_definition'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_2_evasion',
                                              technique='definition_aware', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:43<00:00,  1.12s/it]


In [24]:
from sklearn.metrics import f1_score, classification_report

def evaluate_f1_scores(results, valid_labels):
    """
    Parses LLM predictions from a list of result dictionaries and calculates F1-scores.
    """
    y_true = []
    y_pred = []
    
    for row in results:
        ground_truth = row['ground_truth']
        raw_prediction = row['prediction']
        
        # 1. Safely Extract the Label
        # We check if any of the valid labels are contained within the generated text.
        # This prevents errors if the model outputs "The label is: Deflection."
        parsed_pred = "Unknown" # Fallback if the model hallucinated
        for label in valid_labels:
            if label.lower() in raw_prediction.lower():
                parsed_pred = label
                break
                
        y_true.append(ground_truth)
        y_pred.append(parsed_pred)

    # 2. Calculate F1 Scores
    # zero_division=0 prevents warnings if a class was never predicted
    macro_f1 = f1_score(y_true, y_pred, average='macro', labels=valid_labels, zero_division=0)
    weighted_f1 = f1_score(y_true, y_pred, average='weighted', labels=valid_labels, zero_division=0)
    
    print(f"Macro F1-Score:    {macro_f1:.4f}")
    print(f"Weighted F1-Score: {weighted_f1:.4f}\n")
    
    # 3. Generate the Classification Report
    # This builds a text report showing precision, recall, and F1 for each class.
    print("--- Detailed Classification Report ---")
    print(classification_report(y_true, y_pred, labels=valid_labels, zero_division=0))
    
    return macro_f1, weighted_f1

# --- How to use it with your data ---



# Or for evasion:
# print("Evaluating Evasion Task:")
# evaluate_f1_scores(evasion_results, evasion_labels)

In [42]:
for i in range(10):
    print(subset[i]['annotator1'], subset[i]['annotator3'], subset[i]['annotator2'])

Dodging Dodging General
Deflection General General
Explicit Implicit Implicit
Explicit General General
Dodging Dodging Implicit
Dodging Dodging Dodging
Dodging Dodging Dodging
Dodging Dodging Dodging
General General General
Implicit Implicit Implicit


In [43]:
for i in range(10):
    print(res['v2_cot'][i]['ground_truth'])

Dodging
General
Implicit
General
Dodging
Dodging
Dodging
Dodging
General
Implicit


In [47]:
# Define the exact labels your model was trained to output
clarity_labels = ['Clear Reply', 'Clear Non-Reply', 'Ambiguous']
evasion_labels = ['Explicit', 'Implicit', 'Dodging', 'General', 'Deflection', 
                  'Partial/half-answer', 'Declining to answer', 'Claims ignorance', 'Clarification']

# Assuming 'results' is the list returned from your inference loop:
print("Evaluating evasion Task:")
f1 = evaluate_f1_scores(res['v2_cot'], evasion_labels)

Evaluating evasion Task:
Macro F1-Score:    0.0647
Weighted F1-Score: 0.1395

--- Detailed Classification Report ---
                     precision    recall  f1-score   support

           Explicit       0.27      0.94      0.41        79
           Implicit       0.22      0.10      0.14        61
            Dodging       0.33      0.02      0.03        61
            General       0.00      0.00      0.00        55
         Deflection       0.00      0.00      0.00        22
Partial/half-answer       0.00      0.00      0.00         7
Declining to answer       0.00      0.00      0.00        11
   Claims ignorance       0.00      0.00      0.00         8
      Clarification       0.00      0.00      0.00         4

           accuracy                           0.26       308
          macro avg       0.09      0.12      0.06       308
       weighted avg       0.18      0.26      0.14       308



In [63]:
# Assuming 'results' is the list returned from your inference loop:
print("Evaluating evasion Task:")
f1 = evaluate_f1_scores(res['v2_default'], evasion_labels)

Evaluating evasion Task:
Macro F1-Score:    0.0367
Weighted F1-Score: 0.0655

--- Detailed Classification Report ---
                     precision    recall  f1-score   support

           Explicit       0.00      0.00      0.00        79
           Implicit       0.00      0.00      0.00        61
            Dodging       0.20      1.00      0.33        61
            General       0.00      0.00      0.00        55
         Deflection       0.00      0.00      0.00        22
Partial/half-answer       0.00      0.00      0.00         7
Declining to answer       0.00      0.00      0.00        11
   Claims ignorance       0.00      0.00      0.00         8
      Clarification       0.00      0.00      0.00         4

           accuracy                           0.20       308
          macro avg       0.02      0.11      0.04       308
       weighted avg       0.04      0.20      0.07       308



In [74]:
print("Evaluating evasion Task:")
f1 = evaluate_f1_scores(res['v2_persona'], evasion_labels)

Evaluating evasion Task:
Macro F1-Score:    0.0367
Weighted F1-Score: 0.0655

--- Detailed Classification Report ---
                     precision    recall  f1-score   support

           Explicit       0.00      0.00      0.00        79
           Implicit       0.00      0.00      0.00        61
            Dodging       0.20      1.00      0.33        61
            General       0.00      0.00      0.00        55
         Deflection       0.00      0.00      0.00        22
Partial/half-answer       0.00      0.00      0.00         7
Declining to answer       0.00      0.00      0.00        11
   Claims ignorance       0.00      0.00      0.00         8
      Clarification       0.00      0.00      0.00         4

           accuracy                           0.20       308
          macro avg       0.02      0.11      0.04       308
       weighted avg       0.04      0.20      0.07       308



In [25]:
evasion_labels = ['Explicit', 'Implicit', 'Dodging', 'General', 'Deflection', 
                  'Partial/half-answer', 'Declining to answer', 'Claims ignorance', 'Clarification']
print("Evaluating evasion Task:")
f1 = evaluate_f1_scores(res['v2_definition'], evasion_labels)

Evaluating evasion Task:
Macro F1-Score:    0.0825
Weighted F1-Score: 0.1669

--- Detailed Classification Report ---
                     precision    recall  f1-score   support

           Explicit       0.24      0.58      0.34        79
           Implicit       0.22      0.36      0.27        61
            Dodging       0.31      0.08      0.13        61
            General       0.00      0.00      0.00        55
         Deflection       0.00      0.00      0.00        22
Partial/half-answer       0.00      0.00      0.00         7
Declining to answer       0.00      0.00      0.00        11
   Claims ignorance       0.00      0.00      0.00         8
      Clarification       0.00      0.00      0.00         4

           accuracy                           0.24       308
          macro avg       0.09      0.11      0.08       308
       weighted avg       0.17      0.24      0.17       308



In [ ]:
del model, tokenizer